# Writing Your Own Codes

You can use Jupyter for writing Python code however it's not really meant for development. You'll need a text editor. We won't use IDEs like PyCharm for this course. They're usually too heavyweight and complicated and abstract away many details that you should learn about the programming system.

First install a text editor. Sublime Text, Visual Studio Code, Atom, Brackets or the good ol vim (you will have this one by default on most Unix systems). Vim runs in the terminal directly. The others you can configure so that are callable from the commandline. How you can do so depends on the text editor and operating system. You can always open the editor yourself and then open files in it manually, calling from the terminal is just more convenient.

With the text editor installed, you just need to open your Python files with it and write your code. 

DO NOT modify the jupyter notebooks or any other files inside the cloned repository, you will run into problems when pulling updates. Make a new directory with a virtualenv of its own outside of the Git repository and use that for development.

```
# First time setup
mkdir Practice-Directory # make a directory for your own codes
virtualenv -p python3 venv # make virtualenv
cp path/to/Git/repostiory/requirements.txt ./ # copy requirements file
sourcwe venv/bin/activate # activate venv
pip install -r requirements.txt # install requirements
# To create new Python file
touch hamming_distance.py # touch creates a file
# write your code in this file using some editor
python hamming_distance.py # run and debug
```

If you've already modified the repository directory and run into a MERGE CONFLICT error when pulling from Github, run:

```
git add -A
git commit -m "mergin local changes."
git pull origin master
```

This commits your local changes so that new changes can be applied on top of them.

# Neighborhood Generation

Last week we looked at how to generate neighborhood of distance $1$ for a string. In the quiz, you were asked to calculated the size of the neighborhood for $d = 2$. Let's go over than again.

With a kmer of size $k$ distance of $d$, there are ${k \choose d}$ ways we can select exactly $d$
characters to change. Once we've selected the $d$ characters to change, we want to make sure that all $d$ characters are changed. Each can be modified to one of the other 3 possible bases. So there will be ${k \choose d} \times d^3$ possible strings.

Now how do we generate these programatically?

We can use nested `for` loops. For $d = 1$, we only need one loop, for $d = 2$ we'll need two nested loops:

In [65]:
def generate_neighborhood_2(seq):
    neighborhood = {}
    bases = ['A', 'C', 'G', 'T']
    for i in range(len(seq)): # select first position
        for d in bases:
            if seq[i] != d:
                for j in range(i + 1, len(seq)): # select second position
                    for c in bases:
                        if seq[j] != c:
                            tmp = seq[:i] + d + seq[i + 1:j] + c + seq[j + 1:]
                            neighborhood[tmp] = True
    neighborhood[seq] = True
    print(len(neighborhood))
    return neighborhood

In [66]:
generate_neighborhood_2('GATG')

55


{'ACTG': True,
 'AGTG': True,
 'ATTG': True,
 'AAAG': True,
 'AACG': True,
 'AAGG': True,
 'AATA': True,
 'AATC': True,
 'AATT': True,
 'CCTG': True,
 'CGTG': True,
 'CTTG': True,
 'CAAG': True,
 'CACG': True,
 'CAGG': True,
 'CATA': True,
 'CATC': True,
 'CATT': True,
 'TCTG': True,
 'TGTG': True,
 'TTTG': True,
 'TAAG': True,
 'TACG': True,
 'TAGG': True,
 'TATA': True,
 'TATC': True,
 'TATT': True,
 'GCAG': True,
 'GCCG': True,
 'GCGG': True,
 'GCTA': True,
 'GCTC': True,
 'GCTT': True,
 'GGAG': True,
 'GGCG': True,
 'GGGG': True,
 'GGTA': True,
 'GGTC': True,
 'GGTT': True,
 'GTAG': True,
 'GTCG': True,
 'GTGG': True,
 'GTTA': True,
 'GTTC': True,
 'GTTT': True,
 'GAAA': True,
 'GAAC': True,
 'GAAT': True,
 'GACA': True,
 'GACC': True,
 'GACT': True,
 'GAGA': True,
 'GAGC': True,
 'GAGT': True,
 'GATG': True}

Now you may be thinking to just add three loops for $d = 3$, but this is not scalable. How can we write one function that does it for any $d$?

In [68]:
bases = ['A', 'C', 'G', 'T']

def generate_neighborhood(kmer, seq, i, d, neighborhood):
    print(i, d, seq)
    if i > len(kmer):
        pass
    elif d == 0:
        tmp = seq + kmer[i:]
        #print(tmp)
        neighborhood[tmp] = True
    else:
        for j in range(i, len(kmer)):
            #print(j)
            for base in bases:
                if kmer[j] != base:
                    tmp = seq + kmer[i: j] + base
                    generate_neighborhood(kmer, tmp, j + 1, d - 1, neighborhood)

def generate_neighborhood_d(kmer, d):
    neighborhood = {}
    generate_neighborhood(kmer, '', 0, d, neighborhood)
    print(len(neighborhood))

In [69]:
generate_neighborhood_d('ATCG', 3)

0 3 
1 2 C
2 1 CA
3 0 CAA
3 0 CAG
3 0 CAT
4 0 CACA
4 0 CACC
4 0 CACT
2 1 CC
3 0 CCA
3 0 CCG
3 0 CCT
4 0 CCCA
4 0 CCCC
4 0 CCCT
2 1 CG
3 0 CGA
3 0 CGG
3 0 CGT
4 0 CGCA
4 0 CGCC
4 0 CGCT
3 1 CTA
4 0 CTAA
4 0 CTAC
4 0 CTAT
3 1 CTG
4 0 CTGA
4 0 CTGC
4 0 CTGT
3 1 CTT
4 0 CTTA
4 0 CTTC
4 0 CTTT
4 1 CTCA
4 1 CTCC
4 1 CTCT
1 2 G
2 1 GA
3 0 GAA
3 0 GAG
3 0 GAT
4 0 GACA
4 0 GACC
4 0 GACT
2 1 GC
3 0 GCA
3 0 GCG
3 0 GCT
4 0 GCCA
4 0 GCCC
4 0 GCCT
2 1 GG
3 0 GGA
3 0 GGG
3 0 GGT
4 0 GGCA
4 0 GGCC
4 0 GGCT
3 1 GTA
4 0 GTAA
4 0 GTAC
4 0 GTAT
3 1 GTG
4 0 GTGA
4 0 GTGC
4 0 GTGT
3 1 GTT
4 0 GTTA
4 0 GTTC
4 0 GTTT
4 1 GTCA
4 1 GTCC
4 1 GTCT
1 2 T
2 1 TA
3 0 TAA
3 0 TAG
3 0 TAT
4 0 TACA
4 0 TACC
4 0 TACT
2 1 TC
3 0 TCA
3 0 TCG
3 0 TCT
4 0 TCCA
4 0 TCCC
4 0 TCCT
2 1 TG
3 0 TGA
3 0 TGG
3 0 TGT
4 0 TGCA
4 0 TGCC
4 0 TGCT
3 1 TTA
4 0 TTAA
4 0 TTAC
4 0 TTAT
3 1 TTG
4 0 TTGA
4 0 TTGC
4 0 TTGT
3 1 TTT
4 0 TTTA
4 0 TTTC
4 0 TTTT
4 1 TTCA
4 1 TTCC
4 1 TTCT
2 2 AA
3 1 AAA
4 0 AAAA
4 0 AAAC
4 0 AAAT
3 1 AAG
4 0 AAGA


# Motif Enumeration

We're trying to find repetitive motifs of a certain lenght $k$ in a DNA sample. However, it's unlikely that all copies of a given motif in the DNA are intact; most will have mutations or errors. How do we solve the problem?

One approach would be to consider all kmers withing distance $d$ of the motif we're looking for and use the solution for the Frequent Words problem. However when $k$ gets large and the number of possible mutations increases, the number of different kmers we need to keep track of in the search grows significantly. The previous solution will not be optimal or practical anymore. 

Assume we allow up to $d$ mismatches, then it can be observed that any kmer we see in the DNA has to be within distance $d$ of the actual motif. This means that we don't need to consider all possible kmers, but only those that are likely to be in the DNA based on what we see.

In [ ]:
def hamming_distance(p, q):
    if len(p) != len(q):
        return -1
    l = len(p)
    d = 0 # hamming distance
    for i in range(l):
        if p[i] != q[i]:
            d += 1
    return d

In [ ]:
def approximate_match(p, q, d):
    if hamming_distance(p, q) <= d:
        return True
    return False

In [ ]:
def find_approximate_matches(text, pattern, d):
    l = len(pattern)
    matches = []
    for i in range(0, len(text) - l + 1):
        if approximate_match(pattern, text[i: i + l], d):
            print(i, text[i: i + l])
            matches.append(text[i: i + l])
    return matches

In [18]:
def motif_enumeration(dnas, k, d):
    patterns = {}
    dna = dnas[0]
    for i in range(len(dna) - k + 1):
        pattern = dna[i: i + k]
        kmer_patterns = {}
        for c in range(1, d + 1):
            kmer_patterns.update(generate_neighborhood_d(pattern, c))
        for kmer_pattern in kmer_patterns:
            found = True
            for dna in dnas: 
                if not find_approximate_matches(dna, kmer_pattern, d):
                    found = False
                    break
            if found:
                patterns[kmer_pattern] = True
    return patterns

# Median String Problem


Simply put, the median string is the kmer that has the smallest summation of errors from all other kmers in the text. If you were to modify every kmer in the DNA to a kmer, the median kmer is the kmer that needs the smallest number of characters changes. Let's try and solve this. The pseudocode is as below:

We need the Hamming Distance code from before:

In [19]:
def hamming_distance(p, q):
    if len(p) != len(q):
        return -1
    l = len(p)
    d = 0 # hamming distance
    for i in range(l):
        if p[i] != q[i]:
            d += 1
    return d

We need to generate all kmers of size $k$, first. We can write this from scratch or use the neighborhood functions from before. How?

Say we have $k = 3$. Start with a random 3-mer `AAA` and find all its neighbors with distance 1, 2 and 3. This gives the set of all kmers of size $3$. Not very efficient though, so let's just generate them all:

In [71]:
def generate_kmers(kmer, k, kmers):
    print(kmer)
    if len(kmer) == k:
        kmers[kmer] = k + 1
    else:
        for base in bases:
            generate_kmers(kmer + base, k, kmers)

def generate_kmers_k(k):
    kmer = ''
    kmers = {}
    for base in bases:
        generate_kmers(kmer + base, k, kmers)
    return kmers

In [ ]:
generate_kmers_k(3)

Now we can write the code for the Median String problem:

In [22]:
def median_string(k, dnas):
    kmers = generate_kmers_k(k)
    #print('We have', len(kmers), ' kmer candidates for the median.')
    for kmer in kmers:
        kmers[kmer] = [k + 1] * len(dnas)
    for d, dna in enumerate(dnas): # for each DNA sequence
        for i in range(len(dna) - k + 1): # enumerate kmers
            kmer = dna[i: i + k]
            for pattern in kmers: # calculate hamming distance with each pattern
                kmers[pattern][d] = min(kmers[pattern][d], hamming_distance(pattern, kmer))
    #mean_kmer = min(kmers, key = lambda kmer: sum(kmers[kmer]))
    mean_kmer = None
    mean_distance = 100000
    for kmer in kmers:
        distance = sum(kmers[kmer])
        if distance < mean_distance:
            distance = mean_distance
            mean_kmer = kmer
    # check if there are multiple answers
    d = sum(kmers[mean_kmer])
    for kmer in kmers:
        if sum(kmers[kmer]) == d:
            print(kmer)

In [23]:
dnas = [
    'AAATTGACGCAT',
    'GACGACCACGTT',
    'CGTCAGCGCCTG',
    'GCTGAGCACCGG',
    'AGTACGGGACAG'
]
median_string(3, dnas)

We have 64  kmer candidates for the median.
ACG
GAC


What is the complexity of the above code?

Assuming we have $d$ DNA sequences each with length $n$, how many hamming distances are calculated?

$(n - k) * d * 4 ^k$

So the algorithm is linear in the size of the DNA sequences assuming a fixed $k$. However the $4 ^ k$ coefficient poses a huge problem.

Let's see why. Let's try a new test data set:

In [73]:
dna = 'GACGACCACGTTGACGACCACGTTCGTCAGCGCCTGGCTGAGCACCGGAGTACGGGACAGGACGACCACGTTGACGACCACGTTCGTCAGCGCCTGGCTGAGCACCGGAGTACGGGACAG'
dnas = [dna] * 5 # just make 5 copies
median_string(15, dnas)


AAAAACATACTTTGG
AAAAACATACTTTGT
AAAAACATACTTTT
AAAAACATACTTTTA
AAAAACATACTTTTC
AAAAACATACTTTTG
AAAAACATACTTTTT
AAAAACATAG
AAAAACATAGA
AAAAACATAGAA
AAAAACATAGAAA
AAAAACATAGAAAA
AAAAACATAGAAAAA
AAAAACATAGAAAAC
AAAAACATAGAAAAG
AAAAACATAGAAAAT
AAAAACATAGAAAC
AAAAACATAGAAACA
AAAAACATAGAAACC
AAAAACATAGAAACG
AAAAACATAGAAACT
AAAAACATAGAAAG
AAAAACATAGAAAGA
AAAAACATAGAAAGC
AAAAACATAGAAAGG
AAAAACATAGAAAGT
AAAAACATAGAAAT
AAAAACATAGAAATA
AAAAACATAGAAATC
AAAAACATAGAAATG
AAAAACATAGAAATT
AAAAACATAGAAC
AAAAACATAGAACA
AAAAACATAGAACAA
AAAAACATAGAACAC
AAAAACATAGAACAG
AAAAACATAGAACAT
AAAAACATAGAACC
AAAAACATAGAACCA
AAAAACATAGAACCC
AAAAACATAGAACCG
AAAAACATAGAACCT
AAAAACATAGAACG
AAAAACATAGAACGA
AAAAACATAGAACGC
AAAAACATAGAACGG
AAAAACATAGAACGT
AAAAACATAGAACT
AAAAACATAGAACTA
AAAAACATAGAACTC
AAAAACATAGAACTG
AAAAACATAGAACTT
AAAAACATAGAAG
AAAAACATAGAAGA
AAAAACATAGAAGAA
AAAAACATAGAAGAC
AAAAACATAGAAGAG
AAAAACATAGAAGAT
AAAAACATAGAAGC
AAAAACATAGAAGCA
AAAAACATAGAAGCC
AAAAACATAGAAGCG
AAAAACATAGAAGCT
AAAAACATAGAAGG
AAAAA

AAAAACATAGTTGAG
AAAAACATAGTTGAT
AAAAACATAGTTGC
AAAAACATAGTTGCA
AAAAACATAGTTGCC
AAAAACATAGTTGCG
AAAAACATAGTTGCT
AAAAACATAGTTGG
AAAAACATAGTTGGA
AAAAACATAGTTGGC
AAAAACATAGTTGGG
AAAAACATAGTTGGT
AAAAACATAGTTGT
AAAAACATAGTTGTA
AAAAACATAGTTGTC
AAAAACATAGTTGTG
AAAAACATAGTTGTT
AAAAACATAGTTT
AAAAACATAGTTTA
AAAAACATAGTTTAA
AAAAACATAGTTTAC
AAAAACATAGTTTAG
AAAAACATAGTTTAT
AAAAACATAGTTTC
AAAAACATAGTTTCA
AAAAACATAGTTTCC
AAAAACATAGTTTCG
AAAAACATAGTTTCT
AAAAACATAGTTTG
AAAAACATAGTTTGA
AAAAACATAGTTTGC
AAAAACATAGTTTGG
AAAAACATAGTTTGT
AAAAACATAGTTTT
AAAAACATAGTTTTA
AAAAACATAGTTTTC
AAAAACATAGTTTTG
AAAAACATAGTTTTT
AAAAACATAT
AAAAACATATA
AAAAACATATAA
AAAAACATATAAA
AAAAACATATAAAA
AAAAACATATAAAAA
AAAAACATATAAAAC
AAAAACATATAAAAG
AAAAACATATAAAAT
AAAAACATATAAAC
AAAAACATATAAACA
AAAAACATATAAACC
AAAAACATATAAACG
AAAAACATATAAACT
AAAAACATATAAAG
AAAAACATATAAAGA
AAAAACATATAAAGC
AAAAACATATAAAGG
AAAAACATATAAAGT
AAAAACATATAAAT
AAAAACATATAAATA
AAAAACATATAAATC
AAAAACATATAAATG
AAAAACATATAAATT
AAAAACATATAAC
AAAAACATATAACA
AAAAAC

AAAAACATATTGTT
AAAAACATATTGTTA
AAAAACATATTGTTC
AAAAACATATTGTTG
AAAAACATATTGTTT
AAAAACATATTT
AAAAACATATTTA
AAAAACATATTTAA
AAAAACATATTTAAA
AAAAACATATTTAAC
AAAAACATATTTAAG
AAAAACATATTTAAT
AAAAACATATTTAC
AAAAACATATTTACA
AAAAACATATTTACC
AAAAACATATTTACG
AAAAACATATTTACT
AAAAACATATTTAG
AAAAACATATTTAGA
AAAAACATATTTAGC
AAAAACATATTTAGG
AAAAACATATTTAGT
AAAAACATATTTAT
AAAAACATATTTATA
AAAAACATATTTATC
AAAAACATATTTATG
AAAAACATATTTATT
AAAAACATATTTC
AAAAACATATTTCA
AAAAACATATTTCAA
AAAAACATATTTCAC
AAAAACATATTTCAG
AAAAACATATTTCAT
AAAAACATATTTCC
AAAAACATATTTCCA
AAAAACATATTTCCC
AAAAACATATTTCCG
AAAAACATATTTCCT
AAAAACATATTTCG
AAAAACATATTTCGA
AAAAACATATTTCGC
AAAAACATATTTCGG
AAAAACATATTTCGT
AAAAACATATTTCT
AAAAACATATTTCTA
AAAAACATATTTCTC
AAAAACATATTTCTG
AAAAACATATTTCTT
AAAAACATATTTG
AAAAACATATTTGA
AAAAACATATTTGAA
AAAAACATATTTGAC
AAAAACATATTTGAG
AAAAACATATTTGAT
AAAAACATATTTGC
AAAAACATATTTGCA
AAAAACATATTTGCC
AAAAACATATTTGCG
AAAAACATATTTGCT
AAAAACATATTTGG
AAAAACATATTTGGA
AAAAACATATTTGGC
AAAAACATATTTGGG
AAAAACATATTTG

AAAAACATCCAAC
AAAAACATCCAACA
AAAAACATCCAACAA
AAAAACATCCAACAC
AAAAACATCCAACAG
AAAAACATCCAACAT
AAAAACATCCAACC
AAAAACATCCAACCA
AAAAACATCCAACCC
AAAAACATCCAACCG
AAAAACATCCAACCT
AAAAACATCCAACG
AAAAACATCCAACGA
AAAAACATCCAACGC
AAAAACATCCAACGG
AAAAACATCCAACGT
AAAAACATCCAACT
AAAAACATCCAACTA
AAAAACATCCAACTC
AAAAACATCCAACTG
AAAAACATCCAACTT
AAAAACATCCAAG
AAAAACATCCAAGA
AAAAACATCCAAGAA
AAAAACATCCAAGAC
AAAAACATCCAAGAG
AAAAACATCCAAGAT
AAAAACATCCAAGC
AAAAACATCCAAGCA
AAAAACATCCAAGCC
AAAAACATCCAAGCG
AAAAACATCCAAGCT
AAAAACATCCAAGG
AAAAACATCCAAGGA
AAAAACATCCAAGGC
AAAAACATCCAAGGG
AAAAACATCCAAGGT
AAAAACATCCAAGT
AAAAACATCCAAGTA
AAAAACATCCAAGTC
AAAAACATCCAAGTG
AAAAACATCCAAGTT
AAAAACATCCAAT
AAAAACATCCAATA
AAAAACATCCAATAA
AAAAACATCCAATAC
AAAAACATCCAATAG
AAAAACATCCAATAT
AAAAACATCCAATC
AAAAACATCCAATCA
AAAAACATCCAATCC
AAAAACATCCAATCG
AAAAACATCCAATCT
AAAAACATCCAATG
AAAAACATCCAATGA
AAAAACATCCAATGC
AAAAACATCCAATGG
AAAAACATCCAATGT
AAAAACATCCAATT
AAAAACATCCAATTA
AAAAACATCCAATTC
AAAAACATCCAATTG
AAAAACATCCAATTT
AAAAACATCC

AAAAACATCCTGTGA
AAAAACATCCTGTGC
AAAAACATCCTGTGG
AAAAACATCCTGTGT
AAAAACATCCTGTT
AAAAACATCCTGTTA
AAAAACATCCTGTTC
AAAAACATCCTGTTG
AAAAACATCCTGTTT
AAAAACATCCTT
AAAAACATCCTTA
AAAAACATCCTTAA
AAAAACATCCTTAAA
AAAAACATCCTTAAC
AAAAACATCCTTAAG
AAAAACATCCTTAAT
AAAAACATCCTTAC
AAAAACATCCTTACA
AAAAACATCCTTACC
AAAAACATCCTTACG
AAAAACATCCTTACT
AAAAACATCCTTAG
AAAAACATCCTTAGA
AAAAACATCCTTAGC
AAAAACATCCTTAGG
AAAAACATCCTTAGT
AAAAACATCCTTAT
AAAAACATCCTTATA
AAAAACATCCTTATC
AAAAACATCCTTATG
AAAAACATCCTTATT
AAAAACATCCTTC
AAAAACATCCTTCA
AAAAACATCCTTCAA
AAAAACATCCTTCAC
AAAAACATCCTTCAG
AAAAACATCCTTCAT
AAAAACATCCTTCC
AAAAACATCCTTCCA
AAAAACATCCTTCCC
AAAAACATCCTTCCG
AAAAACATCCTTCCT
AAAAACATCCTTCG
AAAAACATCCTTCGA
AAAAACATCCTTCGC
AAAAACATCCTTCGG
AAAAACATCCTTCGT
AAAAACATCCTTCT
AAAAACATCCTTCTA
AAAAACATCCTTCTC
AAAAACATCCTTCTG
AAAAACATCCTTCTT
AAAAACATCCTTG
AAAAACATCCTTGA
AAAAACATCCTTGAA
AAAAACATCCTTGAC
AAAAACATCCTTGAG
AAAAACATCCTTGAT
AAAAACATCCTTGC
AAAAACATCCTTGCA
AAAAACATCCTTGCC
AAAAACATCCTTGCG
AAAAACATCCTTGCT
AAAAACATCCTT

...

AAAAACATTAATTGC
AAAAACATTAATTGG
AAAAACATTAATTGT
AAAAACATTAATTT
AAAAACATTAATTTA
AAAAACATTAATTTC
AAAAACATTAATTTG
AAAAACATTAATTTT
AAAAACATTAC
AAAAACATTACA
AAAAACATTACAA
AAAAACATTACAAA
AAAAACATTACAAAA
AAAAACATTACAAAC
AAAAACATTACAAAG
AAAAACATTACAAAT
AAAAACATTACAAC
AAAAACATTACAACA
AAAAACATTACAACC
AAAAACATTACAACG
AAAAACATTACAACT
AAAAACATTACAAG
AAAAACATTACAAGA
AAAAACATTACAAGC
AAAAACATTACAAGG
AAAAACATTACAAGT
AAAAACATTACAAT
AAAAACATTACAATA
AAAAACATTACAATC
AAAAACATTACAATG
AAAAACATTACAATT
AAAAACATTACAC
AAAAACATTACACA
AAAAACATTACACAA
AAAAACATTACACAC
AAAAACATTACACAG
AAAAACATTACACAT
AAAAACATTACACC
AAAAACATTACACCA
AAAAACATTACACCC
AAAAACATTACACCG
AAAAACATTACACCT
AAAAACATTACACG
AAAAACATTACACGA
AAAAACATTACACGC
AAAAACATTACACGG
AAAAACATTACACGT
AAAAACATTACACT
AAAAACATTACACTA
AAAAACATTACACTC
AAAAACATTACACTG
AAAAACATTACACTT
AAAAACATTACAG
AAAAACATTACAGA
AAAAACATTACAGAA
AAAAACATTACAGAC
AAAAACATTACAGAG
AAAAACATTACAGAT
AAAAACATTACAGC
AAAAACATTACAGCA
AAAAACATTACAGCC
AAAAACATTACAGCG
AAAAACATTACAGCT
AAAAACATTACAGG
A

AAAAACATTCCCCTG
AAAAACATTCCCCTT
AAAAACATTCCCG
AAAAACATTCCCGA
AAAAACATTCCCGAA
AAAAACATTCCCGAC
AAAAACATTCCCGAG
AAAAACATTCCCGAT
AAAAACATTCCCGC
AAAAACATTCCCGCA
AAAAACATTCCCGCC
AAAAACATTCCCGCG
AAAAACATTCCCGCT
AAAAACATTCCCGG
AAAAACATTCCCGGA
AAAAACATTCCCGGC
AAAAACATTCCCGGG
AAAAACATTCCCGGT
AAAAACATTCCCGT
AAAAACATTCCCGTA
AAAAACATTCCCGTC
AAAAACATTCCCGTG
AAAAACATTCCCGTT
AAAAACATTCCCT
AAAAACATTCCCTA
AAAAACATTCCCTAA
AAAAACATTCCCTAC
AAAAACATTCCCTAG
AAAAACATTCCCTAT
AAAAACATTCCCTC
AAAAACATTCCCTCA
AAAAACATTCCCTCC
AAAAACATTCCCTCG
AAAAACATTCCCTCT
AAAAACATTCCCTG
AAAAACATTCCCTGA
AAAAACATTCCCTGC
AAAAACATTCCCTGG
AAAAACATTCCCTGT
AAAAACATTCCCTT
AAAAACATTCCCTTA
AAAAACATTCCCTTC
AAAAACATTCCCTTG
AAAAACATTCCCTTT
AAAAACATTCCG
AAAAACATTCCGA
AAAAACATTCCGAA
AAAAACATTCCGAAA
AAAAACATTCCGAAC
AAAAACATTCCGAAG
AAAAACATTCCGAAT
AAAAACATTCCGAC
AAAAACATTCCGACA
AAAAACATTCCGACC
AAAAACATTCCGACG
AAAAACATTCCGACT
AAAAACATTCCGAG
AAAAACATTCCGAGA
AAAAACATTCCGAGC
AAAAACATTCCGAGG
AAAAACATTCCGAGT
AAAAACATTCCGAT
AAAAACATTCCGATA
AAAAACATTCCGA

AAAAACATTGCTAAG
AAAAACATTGCTAAT
AAAAACATTGCTAC
AAAAACATTGCTACA
AAAAACATTGCTACC
AAAAACATTGCTACG
AAAAACATTGCTACT
AAAAACATTGCTAG
AAAAACATTGCTAGA
AAAAACATTGCTAGC
AAAAACATTGCTAGG
AAAAACATTGCTAGT
AAAAACATTGCTAT
AAAAACATTGCTATA
AAAAACATTGCTATC
AAAAACATTGCTATG
AAAAACATTGCTATT
AAAAACATTGCTC
AAAAACATTGCTCA
AAAAACATTGCTCAA
AAAAACATTGCTCAC
AAAAACATTGCTCAG
AAAAACATTGCTCAT
AAAAACATTGCTCC
AAAAACATTGCTCCA
AAAAACATTGCTCCC
AAAAACATTGCTCCG
AAAAACATTGCTCCT
AAAAACATTGCTCG
AAAAACATTGCTCGA
AAAAACATTGCTCGC
AAAAACATTGCTCGG
AAAAACATTGCTCGT
AAAAACATTGCTCT
AAAAACATTGCTCTA
AAAAACATTGCTCTC
AAAAACATTGCTCTG
AAAAACATTGCTCTT
AAAAACATTGCTG
AAAAACATTGCTGA
AAAAACATTGCTGAA
AAAAACATTGCTGAC
AAAAACATTGCTGAG
AAAAACATTGCTGAT
AAAAACATTGCTGC
AAAAACATTGCTGCA
AAAAACATTGCTGCC
AAAAACATTGCTGCG
AAAAACATTGCTGCT
AAAAACATTGCTGG
AAAAACATTGCTGGA
AAAAACATTGCTGGC
AAAAACATTGCTGGG
AAAAACATTGCTGGT
AAAAACATTGCTGT
AAAAACATTGCTGTA
AAAAACATTGCTGTC
AAAAACATTGCTGTG
AAAAACATTGCTGTT
AAAAACATTGCTT
AAAAACATTGCTTA
AAAAACATTGCTTAA
AAAAACATTGCTTAC
AAAAACATTG

AAAAACATTTGAGG
AAAAACATTTGAGGA
AAAAACATTTGAGGC
AAAAACATTTGAGGG
AAAAACATTTGAGGT
AAAAACATTTGAGT
AAAAACATTTGAGTA
AAAAACATTTGAGTC
AAAAACATTTGAGTG
AAAAACATTTGAGTT
AAAAACATTTGAT
AAAAACATTTGATA
AAAAACATTTGATAA
AAAAACATTTGATAC
AAAAACATTTGATAG
AAAAACATTTGATAT
AAAAACATTTGATC
AAAAACATTTGATCA
AAAAACATTTGATCC
AAAAACATTTGATCG
AAAAACATTTGATCT
AAAAACATTTGATG
AAAAACATTTGATGA
AAAAACATTTGATGC
AAAAACATTTGATGG
AAAAACATTTGATGT
AAAAACATTTGATT
AAAAACATTTGATTA
AAAAACATTTGATTC
AAAAACATTTGATTG
AAAAACATTTGATTT
AAAAACATTTGC
AAAAACATTTGCA
AAAAACATTTGCAA
AAAAACATTTGCAAA
AAAAACATTTGCAAC
AAAAACATTTGCAAG
AAAAACATTTGCAAT
AAAAACATTTGCAC
AAAAACATTTGCACA
AAAAACATTTGCACC
AAAAACATTTGCACG
AAAAACATTTGCACT
AAAAACATTTGCAG
AAAAACATTTGCAGA
AAAAACATTTGCAGC
AAAAACATTTGCAGG
AAAAACATTTGCAGT
AAAAACATTTGCAT
AAAAACATTTGCATA
AAAAACATTTGCATC
AAAAACATTTGCATG
AAAAACATTTGCATT
AAAAACATTTGCC
AAAAACATTTGCCA
AAAAACATTTGCCAA
AAAAACATTTGCCAC
AAAAACATTTGCCAG
AAAAACATTTGCCAT
AAAAACATTTGCCC
AAAAACATTTGCCCA
AAAAACATTTGCCCC
AAAAACATTTGCCCG
AAAAACATTTGCC

KeyboardInterrupt: 

See how long it takes? The number of possible candidates grows exponentially. Anything that grows exponentially should better be avoided in computer science.

# Greedy Motif Search

So instead of trying every possible kmer, let's just look at the ones already in the DNA sequences and find something. We use the profile matrix to see which sequences is more likely.

In [ ]:
GreedyMotifSearch(Dna, k, t)
    BestMotifs ← motif matrix formed by first k-mers in each string from Dna
    for each k-mer Motif in the first string from Dna
        Motif1 ← Motif
        for i = 2 to t
            form Profile from motifs Motif1, …, Motifi - 1
            Motifi ← Profile-most probable k-mer in the i-th string in Dna
        Motifs ← (Motif1, …, Motift)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs

How to implement this?

First create profile matrix. One row for each base and on column for each $0 \leq i < k$. First we count the number of time each base is seen in each column then we divide that by the number of motifs to get a probability.

In [22]:
bases = ['A', 'C', 'G', 'T']

0.5

def create_profile_matrix(motifs, k):
    matrix = {base: [0] * k  for base in bases}
    for motif in motifs:
        for i, c in enumerate(motif):
            matrix[c][i] += 1
    for base in bases:
        for i in range(k):
            if matrix[base][i] == 0:
                matrix[base][i] = 1
            matrix[base][i] = (matrix[base][i] + 0.5) / (len(motifs) + (4 * 0.5))
    return matrix

In [23]:
dnas = [
    'GGC',
    'AAG',
    'CAA',
    'CAC',
    'CAA'
]

matrix = create_profile_matrix(dnas, 3)

matrix

{'A': [0.1111111111111111, 0.4444444444444444, 0.2222222222222222],
 'C': [0.3333333333333333, 0.0, 0.2222222222222222],
 'G': [0.1111111111111111, 0.1111111111111111, 0.1111111111111111],
 'T': [0.0, 0.0, 0.0]}

In [24]:
def calculate_probability(kmer, matrix):
    prob = 1.0
    for i, c in enumerate(kmer):
        prob *= matrix[c][i]
    return prob

In [25]:
calculate_probability('TTC', matrix)
calculate_probability('TAA', matrix)

0.0

In [26]:
def score_profile(matrix, k):
    score = 0
    for i in range(k):
        score += max([matrix[base][i] for base in bases])
    return score

In [27]:
score_profile(matrix, 3)


0.9999999999999999

In [ ]:
GreedyMotifSearch(Dna, k, t)
    BestMotifs ← motif matrix formed by first k-mers in each string from Dna
    for each k-mer Motif in the first string from Dna
        Motif1 ← Motif
        for i = 2 to t
            form Profile from motifs Motif1, …, Motifi - 1
            Motifi ← Profile-most probable k-mer in the i-th string in Dna
        Motifs ← (Motif1, …, Motift)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs

In [31]:
def greedy_motifs(dnas, k, t):
    best_motifs = [dna[0:k] for dna in dnas]
    best_profile = create_profile_matrix(best_motifs, k)
    print(best_profile)
    motifs = []
    for i in range(len(dnas[0]) - k + 1):
        motif = dnas[0][i:i + k]
        motifs.append(motif)
        profile = create_profile_matrix(motifs, k)
        for j in range(1, t):
            #print(profile)
            max_prob = -1
            next_motif = None
            for h in range(0, len(dnas[j]) - k + 1):
                kmer = dnas[j][h:h + k]
                prob = calculate_probability(kmer, profile)
                if prob > max_prob:
                    max_prob = prob
                    next_motif = kmer
            #print(next_motif)
            motifs.append(next_motif)
            profile = create_profile_matrix(motifs, k)
        if score_profile(best_profile, k) < score_profile(profile, k):
            best_motifs = motifs
            best_profile = profile
        motifs = []
    return best_motifs

In [33]:
dnas = [
    'GGCGTTCAGGCA',
    'AAGAATCAGTCA',
    'CAAGGAGTTCGC',
    'CACGTCAATCAC',
    'CAATAATATTCG'
]
greedy_motifs(dnas, 3, 5)

{'A': [0.1111111111111111, 0.4444444444444444, 0.2222222222222222], 'C': [0.3333333333333333, 0.0, 0.2222222222222222], 'G': [0.1111111111111111, 0.1111111111111111, 0.1111111111111111], 'T': [0.0, 0.0, 0.0]}


['CAG', 'CAG', 'CAA', 'CAA', 'CAA']